In [1]:
import gradio as gr
import requests
import fitz
import docx2txt
import pandas as pd
import json
import os

c:\Users\anjan\OneDrive\Desktop\hand_to_text\.htd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nest_asyncio
nest_asyncio.apply()


In [3]:
def extract_file(file):
    fn = file.name.lower()
    try:
        if fn.endswith(".pdf"):
            pdf_file = fitz.open(fn)
            text = ""
            for page in pdf_file:
                text += page.get_text()
            return text
        elif fn.endswith(".docx"):
            return docx2txt.process(fn)
        elif fn.endswith(".txt"):
            with open(fn, "r", encoding="utf-8") as f:
                return f.read()
        else:
            return "Unsupported file format. Please upload PDF, DOCX, or TXT."
    except Exception as e:
        return f"Error reading file: {e}"


In [ ]:
# def generate_mcqs(content, n=3, level="Easy"):
#     prompt = f"""
# Generate {n} single-choice MCQs based on the text below.
# Difficulty level: {level}

# Instructions:
# - Each question should have exactly 4 options (a–d).
# - Only one option should be correct.
# - Provide answers and short explanations.
# - Don't provide anything off topic from the content given by the user 
# - Use this format strictly:

# Q1. <question>
# a) <option1>
# b) <option2>
# c) <option3>
# d) <option4>
# Answer: <correct option letter>
# Explanation: <short explanation>

# Text:
# ---
# {content}
# ---
# """
#     response = requests.post(
#         "http://localhost:11434/api/generate",
#         json={"model": "phi3", "prompt": prompt, "stream": False}
#     )
#     return response.json().get("response", "").strip()


Single choice

In [14]:
def generate_single_choice(content, n=3, level="Easy"):
    prompt = f"""
Generate {n} single-choice MCQs based on the text below.
Difficulty level: {level}

Instructions:
- Each question should have exactly 4 options (a–d).
- Only one option should be correct.
- Provide answers and short explanations.
- Stay strictly within the topic of the text.
- Use this format strictly:

Q1. <question>
a) <option1>
b) <option2>
c) <option3>
d) <option4>
Answer: <correct option letter>
Explanation: <short explanation>

Text:
---
{content}
---
"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "phi3", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "").strip()


Multi choice

In [15]:
def generate_multi_choice(content, n=3, level="Medium"):
    prompt = f"""
Generate {n} multiple-choice questions based on the text below.
Difficulty level: {level}

Instructions:
- Each question should have 4 options (a–d).
- Two or more options can be correct.
- List all correct options (e.g., "Answer: a, c").
- Provide a short explanation for why those are correct.
- Stay strictly within the topic of the text.

Format strictly:

Q1. <question>
a) <option1>
b) <option2>
c) <option3>
d) <option4>
Answer: <correct option letters>
Explanation: <short explanation>

Text:
---
{content}
---
"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "phi3", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "").strip()


True /False

In [ ]:
def generate_true_false(content, n=5, level="Easy"):
    prompt = f"""
Generate {n} True or False questions based on the text below.
Difficulty level: {level}

Instructions:
- Write concise factual statements that can be clearly True or False.
- Provide the correct answer and a short explanation.
- Stay relevant to the text only.
- Stay strictly within the topic of the text.

Format strictly:

Q1. <statement>
a) True
b) False
Answer: True or False
Explanation: <short explanation>

Text:
---
{content}
---
"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "phi3", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "").strip()


Fill in the Blannks

In [34]:
def generate_fill_blanks(content, n=5, level="Medium"):
    prompt = f"""
You are an educational content expert. Generate {n} meaningful fill-in-the-blank questions based on the text below.
Focus on **concepts, definitions, and key terms**, not literal code symbols or parentheses.

Instructions:
1. Select a key word or phrase that captures the main idea of the sentence.
2. Replace the key word or phrase with a blank (____).
3. Provide a clear, concise answer (the missing word or phrase).
4. Include a short explanation of why this is the correct answer.
5. Do NOT include underscores, parentheses, or code syntax in the blanks unless the concept itself is code-related.
6. Stay strictly within the topic of the text.
Format strictly:
Q1. <sentence with blank>
Answer: <correct word or phrase>
Explanation: <short explanation>

Example:

Q1. ____ is a high-level programming language.
Answer: Python
Explanation: Python is the programming language being described.

Text:
---
{content}
---
"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "phi3", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "").strip()


Subjective

In [18]:
def generate_subjective(content, n=3, level="Hard"):
    prompt = f"""
Generate {n} subjective (open-ended) questions based on the text below.
Difficulty level: {level}

Instructions:
- Each question should test understanding, analysis, or reasoning.
- Do NOT include options.
- Provide a short ideal answer or key points expected in the answer.
- Stay within the topic context.

Format strictly:

Q1. <question>
Ideal Answer: <concise key answer>

Text:
---
{content}
---
"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "phi3", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "").strip()


In [ ]:
# def save_mcqs_to_file(mcq_text, fmt):
#     questions = []
#     for block in mcq_text.split("Q"):
#         block = block.strip()
#         if not block or not block[0].isdigit():
#             continue
#         lines = block.split("\n")
#         q_text = lines[0][2:].strip()
#         options = [l.strip()[3:] for l in lines[1:5] if l.strip().startswith(("a)", "b)", "c)", "d)"))]
#         answer = next((l.split(":")[1].strip() for l in lines if l.startswith("Answer:")), "")
#         explanation = next((l.split(":")[1].strip() for l in lines if l.startswith("Explanation:")), "")
#         questions.append({
#             "Question": q_text,
#             "Option A": options[0] if len(options) > 0 else "",
#             "Option B": options[1] if len(options) > 1 else "",
#             "Option C": options[2] if len(options) > 2 else "",
#             "Option D": options[3] if len(options) > 3 else "",
            
#         })

#     if not questions:
#         return None

#     df = pd.DataFrame(questions)
#     filename = f"mcqs.{fmt}"

#     if fmt == "xlsx":
#         df.to_excel(filename, index=False)
#     elif fmt == "csv":
#         df.to_csv(filename, index=False)
#     elif fmt == "json":
#         df.to_json(filename, orient="records", indent=4)
#     else:
#         return None

#     return os.path.abspath(filename)


In [ ]:
# import pandas as pd
# import os
# import re

# def save_questions_to_file(text, qtype, fmt="xlsx"):
#     questions = []

#     # Split by Q1, Q2... pattern
#     blocks = re.split(r"\n?Q\d+[:.]?", text)

#     for block in blocks:
#         block = block.strip()
#         if not block:
#             continue

#         q_data = {"Type": qtype}

#         # SINGLE / MULTI CHOICE
#         if qtype in ["single", "multi"]:
#             # Split lines
#             lines = [l.strip() for l in block.split("\n") if l.strip()]
#             if not lines:
#                 continue

#             # First line is question
#             q_data["Question"] = lines[0]

#             # Options: all lines until Answer: or Explanation:
#             options = []
#             for l in lines[1:]:
#                 if l.startswith("Answer:") or l.startswith("Explanation:"):
#                     break
#                 options.append(l)

#             # Map options to Option A, B, C...
#             for i, opt in enumerate(options):
#                 q_data[f"Option {chr(65+i)}"] = opt

#             # Answer
#             ans_match = re.search(r"Answer:\s*(.*)", block)
#             q_data["Answer"] = ans_match.group(1).strip() if ans_match else ""

#             # Explanation
#             exp_match = re.search(r"Explanation:\s*(.*)", block)
#             q_data["Explanation"] = exp_match.group(1).strip() if exp_match else ""

#         # TRUE/FALSE
#         elif qtype == "truefalse":
#             q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
#             q_data["Question"] = q_match.group(1).strip() if q_match else block

#             ans_match = re.search(r"Answer:\s*(True|False)", block, re.IGNORECASE)
#             exp_match = re.search(r"Explanation:\s*(.*)", block)
#             q_data["Answer"] = ans_match.group(1).strip().capitalize() if ans_match else ""
#             q_data["Explanation"] = exp_match.group(1).strip() if exp_match else ""

#         # FILL IN THE BLANKS
#         elif qtype == "fill":
#             q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
#             q_data["Question"] = q_match.group(1).strip() if q_match else block

#             ans_match = re.search(r"Answer:\s*(.*)", block)
#             exp_match = re.search(r"Explanation:\s*(.*)", block)
#             q_data["Answer"] = ans_match.group(1).strip() if ans_match else ""
#             q_data["Explanation"] = exp_match.group(1).strip() if exp_match else ""

#         # SUBJECTIVE
#         elif qtype == "subjective":
#             q_match = re.match(r"^(.*?)Ideal Answer:", block, re.DOTALL)
#             q_data["Question"] = q_match.group(1).strip() if q_match else block

#             ans_match = re.search(r"Ideal Answer:\s*(.*)", block)
#             q_data["Ideal Answer"] = ans_match.group(1).strip() if ans_match else ""

#         questions.append(q_data)

#     if not questions:
#         return None

#     df = pd.DataFrame(questions)
#     filename = f"questions_{qtype}.{fmt}"

#     if fmt == "xlsx":
#         df.to_excel(filename, index=False)
#     elif fmt == "csv":
#         df.to_csv(filename, index=False)
#     elif fmt == "json":
#         df.to_json(filename, orient="records", indent=4)
#     else:
#         raise ValueError("Unsupported format. Use 'xlsx', 'csv', or 'json'.")

#     return os.path.abspath(filename)


In [ ]:
# def generate_and_save_mcqs(pasted_text, uploaded_file, n, level, download_format):
#     content = pasted_text or ""
#     if uploaded_file is not None:
#         file_text = extract_file(uploaded_file)
#         if "Unsupported" in file_text or "Error" in file_text:
#             return file_text, None
#         content += "\n" + file_text

#     if not content.strip():
#         return "Please provide text or upload a file.", None

#     mcq_text = generate_mcqs(content, n, level)
#     if not mcq_text.strip():
#         return "No MCQs generated. Please try again.", None

    
#     file_path = save_mcqs_to_file(mcq_text, download_format)
#     return mcq_text.replace("\n", "<br>"), file_path


In [46]:
import pandas as pd
import os
import re

def save_questions_clean(text, qtype="single", fmt="xlsx"):
    questions = []

    # Split by Q1, Q2, ...
    blocks = re.split(r"\n?Q\d+[:.]?", text)

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        q_data = {"Type": qtype}

        if qtype in ["single", "multi"]:
            # Find up to 4 options a-d
            option_matches = list(re.finditer(r"([a-dA-D])\)\s*(.*?)(?=\s*[a-dA-D]\)|Answer:|Explanation:|$)", block, re.DOTALL))
            if not option_matches:
                continue

            # Question is text before first option
            first_option_start = option_matches[0].start()
            question_text = block[:first_option_start].strip()
            # Remove any trailing Answer/Explanation in question
            question_text = re.sub(r"Answer:.*", "", question_text, flags=re.DOTALL).strip()
            question_text = re.sub(r"Explanation:.*", "", question_text, flags=re.DOTALL).strip()
            q_data["Question"] = question_text

            # Initialize 4 options
            for letter in ["A", "B", "C", "D"]:
                q_data[f"Option {letter}"] = ""

            # Fill in options A–D
            for match in option_matches[:4]:
                letter = match.group(1).upper()
                option_text = match.group(2).strip()
                # Remove any [correct]/[incorrect] tags and Explanation
                option_text = re.sub(r"\[.*?\]", "", option_text).strip()
                option_text = re.sub(r"Explanation:.*$", "", option_text).strip()
                q_data[f"Option {letter}"] = option_text

        elif qtype == "truefalse":
            q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
            q_data["Question"] = q_match.group(1).strip() if q_match else block
            q_data["Option A"] = "True"
            q_data["Option B"] = "False"

        elif qtype == "fill":
            q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
            q_data["Question"] = q_match.group(1).strip() if q_match else block
            ans_match = re.search(r"Answer:\s*(.*)", block)
            q_data["Answer"] = ans_match.group(1).strip() if ans_match else ""

        elif qtype == "subjective":
            q_match = re.match(r"^(.*?)Ideal Answer:", block, re.DOTALL)
            q_data["Question"] = q_match.group(1).strip() if q_match else block
            ans_match = re.search(r"Ideal Answer:\s*(.*)", block)
            q_data["Ideal Answer"] = ans_match.group(1).strip() if ans_match else ""

        questions.append(q_data)

    if not questions:
        return None

    # Define explicit columns for MCQs
    if qtype in ["single", "multi"]:
        columns = ["Type", "Question", "Option A", "Option B", "Option C", "Option D"]
    elif qtype == "truefalse":
        columns = ["Type", "Question", "Option A", "Option B"]
    elif qtype == "fill":
        columns = ["Type", "Question", "Answer"]
    elif qtype == "subjective":
        columns = ["Type", "Question", "Ideal Answer"]
    else:
        columns = None

    df = pd.DataFrame(questions)
    if columns:
        df = df.reindex(columns=columns)

    # Default to XLSX if format invalid
    if not fmt or fmt.lower() not in ["xlsx", "csv", "json"]:
        fmt = "xlsx"

    filename = f"questions_{qtype}.{fmt.lower()}"

    if fmt.lower() == "xlsx":
        df.to_excel(filename, index=False)
    elif fmt.lower() == "csv":
        df.to_csv(filename, index=False)
    elif fmt.lower() == "json":
        df.to_json(filename, orient="records", indent=4)

    return os.path.abspath(filename)


In [47]:
def generate_and_save_questions(pasted_text, uploaded_file, n=3, level="Easy", download_format="xlsx", qtype="single"):
    
    content = pasted_text or ""
    if uploaded_file is not None:
        file_text = extract_file(uploaded_file)
        if "Unsupported" in file_text or "Error" in file_text:
            return file_text, None
        content += "\n" + file_text

    if not content.strip():
        return "Please provide text or upload a file.", None

    
    generators = {
        "single": generate_single_choice,
        "multi": generate_multi_choice,
        "truefalse": generate_true_false,
        "fill": generate_fill_blanks,
        "subjective": generate_subjective
    }

    if qtype not in generators:
        return "Invalid question type selected.", None

    mcq_text = generators[qtype](content, n, level)
    if not mcq_text.strip():
        return "No questions generated. Please try again.", None

    
    file_path = save_questions_clean(mcq_text, qtype, download_format)

    
    return mcq_text.replace("\n", "<br>"), file_path


In [ ]:
# iface = gr.Interface(
#     fn=generate_and_save_mcqs,
#     inputs=[
#         gr.Textbox(lines=10, placeholder="Paste your content here..."),
#         gr.File(label="Upload PDF, DOCX, or TXT file"),
#         gr.Number(value=3, precision=0, label="Number of MCQs", interactive=True),
#         gr.Dropdown(["Easy", "Intermediate", "Hard"], value="Easy", label="Difficulty Level"),
#         gr.Dropdown(["xlsx", "csv", "json"], value="xlsx", label="Download Format")
#     ],
#     outputs=[
#         gr.HTML(label="Generated MCQs"),
#         gr.File(label="Download File")
#     ],
#     title=" MCQ Generator",
#     description="Generate single-choice MCQs from text or files using your local Phi-3 model."
# )

# iface.launch(inline=False, share=False)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\anjan\OneDrive\Desktop\hand_to_text\.htd\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\anjan\OneDrive\Desktop\hand_to_text\.htd\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\anjan\OneDrive\Desktop\hand_to_text\.htd\lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\anjan\OneDrive\Desktop\hand_to_text\.htd\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\anjan\OneDrive\Desktop\hand_to_text\.htd\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\anjan\OneDrive\Desktop\hand_to_

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=generate_and_save_questions,
    inputs=[
        gr.Textbox(lines=10, placeholder="Paste your content here..."),
        gr.File(label="Upload PDF, DOCX, or TXT file"),
        gr.Number(value=3, precision=0, label="Number of Questions", interactive=True),
        gr.Dropdown(["Easy", "Intermediate", "Hard"], value="Easy", label="Difficulty Level"),
        gr.Dropdown(["xlsx", "csv", "json"], value="xlsx", label="Download Format"),
        gr.Dropdown(["single", "multi", "truefalse", "fill", "subjective"], value="single", label="Question Type")
    ],
    outputs=[
        gr.HTML(label="Generated Questions"),
        gr.File(label="Download File")
    ],
    title="MCQ / Question Generator",
    description="Generate various types of questions from text or files using your local Phi3 model."
)

iface.launch(inline=False, share=False)


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
